In [1]:
!pip install git+https://github.com/openai/swarm.git
!pip install openai

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-9t4rqx__
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-9t4rqx__
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install httpx==0.23.0

In [3]:
import os
from swarm import Swarm, Agent
from openai import OpenAI
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
api = OpenAI(api_key=" ")

In [5]:
client = Swarm(api)

### Data Import and Cleaning

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
file_path = '/content/drive/MyDrive/ai_agent/sales_data.csv'

# Load the data into a DataFrame
data = pd.read_csv(file_path)

In [8]:
data

,date,source,medium,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue,ad spend
0,2020-05-11,google,organic,no data,PC,no,no,4087,1233,5240,1048,672.00,90,"₱456,877","₱384,039"
1,2020-05-11,facebook,cpc,no data,mobile,yes,no,4326,544,9930,1984,1812.48,217,"₱1,289,066","₱817,514"
2,2020-05-11,google,cpc,no data,mobile,no,no,3891,1450,5460,1090,766.72,100,"₱554,427","₱435,105"
3,2020-05-11,google,cpc,no data,PC,no,no,2456,854,4250,848,520.96,71,"₱416,561","₱635,599"
4,2020-05-11,facebook,organic,no data,PC,no,no,2828,1000,4110,824,449.28,62,"₱326,176","₱428,962"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52716,2020-06-21,(direct),(none),yes,mobile,yes,yes,85,4,250,50,74.24,4,"₱25,420","₱491,731"
52717,2020-08-09,co-promo,email,yes,mobile,no,no,56,2,250,50,29.44,1,"₱38,744","₱433,598"
52718,2020-09-22,google,cpc,yes,mobile,yes,yes,200,3,250,50,74.24,1,"₱5,051","₱198,570"
52719,2020-08-25,(direct),(none),no data,mobile,yes,yes,287,8,250,50,57.60,1,"₱4,366","₱684,341"


## Data Preprocessing

In [9]:
def preprocess_data(data):
    # 1. Convert 'date' column to datetime format
    data['date'] = pd.to_datetime(data['date'], errors='coerce')  # Converts invalid dates to NaT (Not a Time)

    # 2. Clean categorical columns (strip spaces, remove special characters, etc.)
    categorical_columns = ['source', 'medium', 'delivery_available', 'device_type', 'promo_activated', 'filter_used']
    for col in categorical_columns:
        data[col] = data[col].str.strip()  # Remove leading/trailing spaces
        data[col] = data[col].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Remove special characters

    # 3. Convert numeric columns to proper types (force conversion and handle errors)
    numeric_columns = ['pageviews', 'visits', 'productClick', 'addToCart', 'checkout', 'transactions', 'revenue', 'ad spend']
    for col in numeric_columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')  # Convert to numeric, invalid parsing will be set as NaN

    # 4. Handle missing values:
    # - Drop rows with missing 'date' (important for time series analysis)
    # - Fill or drop NaN values for other columns based on context
    data = data.dropna(subset=['date'])  # Keep only rows with valid 'date'

    # Optionally fill missing values in numeric columns (e.g., fill with 0 or median)
    data[numeric_columns] = data[numeric_columns].fillna(0)  # Filling missing numeric values with 0 (or median)

    # 5. Handle duplicates if any
    data = data.drop_duplicates()

    # 6. Check for any outliers or invalid data (optional step, depending on context)
    # Example: If any 'ad spend' is negative or unrealistic, set it to 0
    data['ad spend'] = data['ad spend'].apply(lambda x: max(x, 0))

    return data


In [10]:
data = preprocess_data(data)
data

,date,source,medium,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue,ad spend
0,2020-05-11,google,organic,no data,PC,no,no,4087,1233,5240,1048,672.00,90,0.0,0.0
1,2020-05-11,facebook,cpc,no data,mobile,yes,no,4326,544,9930,1984,1812.48,217,0.0,0.0
2,2020-05-11,google,cpc,no data,mobile,no,no,3891,1450,5460,1090,766.72,100,0.0,0.0
3,2020-05-11,google,cpc,no data,PC,no,no,2456,854,4250,848,520.96,71,0.0,0.0
4,2020-05-11,facebook,organic,no data,PC,no,no,2828,1000,4110,824,449.28,62,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52716,2020-06-21,direct,none,yes,mobile,yes,yes,85,4,250,50,74.24,4,0.0,0.0
52717,2020-08-09,copromo,email,yes,mobile,no,no,56,2,250,50,29.44,1,0.0,0.0
52718,2020-09-22,google,cpc,yes,mobile,yes,yes,200,3,250,50,74.24,1,0.0,0.0
52719,2020-08-25,direct,none,no data,mobile,yes,yes,287,8,250,50,57.60,1,0.0,0.0


## Agent 1: Data Quality Monitoring Agent

In [11]:
import pandas as pd

class DataQualityMonitoringAgent:
    def __init__(self):
        self.name = "Data Quality Monitoring Agent"
        self.model = "gpt-4o-mini"
        self.instructions = (
            """You are a Data Description Analyst that cleans the dataset by removing the ₱ symbol and commas,
            then convert the relevant columns to float data type.
            Afterward, check for missing values, duplicates, data types, and provide a summary of the numerical columns."""
        )

    def clean_data(self, data):
        """
        Clean the dataset by removing the ₱ symbol and commas, then convert relevant columns to float.
        Args:
            data (pd.DataFrame): Input dataset.
        Returns:
            pd.DataFrame: Cleaned dataset.
        """
        # Remove '₱' symbol and commas in relevant columns and convert to float
        relevant_columns = ['ad spend', 'revenue']  # Example columns that may contain currency symbols
        for col in relevant_columns:
            if col in data.columns:
                data[col] = data[col].replace({'₱': '', ',': ''}, regex=True)
                data[col] = pd.to_numeric(data[col], errors='coerce')

        return data

    def check_for_issues(self, data):
        """
        Check for missing values, duplicates, and data types in the dataset.
        Args:
            data (pd.DataFrame): Cleaned dataset.
        Returns:
            dict: A dictionary with issues found.
        """
        # Check for missing values
        missing_values = data.isnull().sum()

        # Check for duplicates
        duplicate_rows = data[data.duplicated()]

        # Check data types
        data_types = data.dtypes

        # Summary of numerical columns
        numeric_summary = data.describe()

        return {
            'missing_values': missing_values,
            'duplicate_rows': duplicate_rows,
            'data_types': data_types,
            'numeric_summary': numeric_summary
        }

    def generate_summary_report(self, issues):
        """
        Generate a summary report in DataFrame format based on the identified issues.
        Args:
            issues (dict): A dictionary containing the issues found.
        Returns:
            pd.DataFrame: A DataFrame with the summary report of the data quality issues.
        """
        # Create DataFrame for missing values
        missing_df = issues['missing_values'].reset_index()
        missing_df.columns = ['Column', 'Missing Values']

        # Create DataFrame for duplicate rows
        duplicate_df = issues['duplicate_rows']

        # Create DataFrame for data types
        data_types_df = issues['data_types'].reset_index()
        data_types_df.columns = ['Column', 'Data Type']

        # Create DataFrame for numeric summary
        numeric_summary_df = issues['numeric_summary'].transpose()

        # Combine all issues into a single report DataFrame
        report = {
            'Missing Values': missing_df,
            'Duplicate Rows': duplicate_df,
            'Data Types': data_types_df,
            'Numeric Summary': numeric_summary_df
        }

        return report

# Initialize the DataQualityMonitoringAgent
agent = DataQualityMonitoringAgent()

# Step 1: Clean the data
cleaned_data = agent.clean_data(data)

# Step 2: Check for issues in the cleaned data
issues = agent.check_for_issues(cleaned_data)

# Step 3: Generate a summary report
report = agent.generate_summary_report(issues)

# Accessing the report for each type of issue
print("Missing Values Report:")
print(report['Missing Values'])

print("\nDuplicate Rows Report:")
print(report['Duplicate Rows'])

print("\nData Types Report:")
print(report['Data Types'])

print("\nNumeric Summary Report:")
print(report['Numeric Summary'])


Missing Values Report:
                Column  Missing Values
0                 date               0
1               source               0
2               medium               0
3   delivery_available               0
4          device_type               0
5      promo_activated               0
6          filter_used               0
7            pageviews               0
8               visits               0
9         productClick               0
10           addToCart               0
11            checkout               0
12        transactions               0
13             revenue               0
14            ad spend               0

Duplicate Rows Report:
Empty DataFrame
Columns: [date, source, medium, delivery_available, device_type, promo_activated, filter_used, pageviews, visits, productClick, addToCart, checkout, transactions, revenue, ad spend]
Index: []

Data Types Report:
                Column       Data Type
0                 date  datetime64[ns]
1               source 

## Agent 2: Data Security Compliance Agent

In [12]:
class DataSecurityComplianceAgent:
    def __init__(self):
        self.name = "Data Security Compliance Agent"
        self.model = "gpt-4o-mini"
        self.instructions = (
            """You are a Data Security Analyst that ensures the dataset is compliant with data security and privacy standards.
            Check for any personally identifiable information (PII) like names, emails, phone numbers, and social security numbers.
            Mask or anonymize sensitive data and ensure that any PII is removed or encrypted. Provide a summary of the compliance check."""
        )

    def check_for_pii(self, data):
        """
        Identify columns that may contain personally identifiable information (PII).
        Args:
            data (pd.DataFrame): Input dataset.
        Returns:
            dict: A dictionary with identified PII columns and the number of occurrences of PII data.
        """
        pii_columns = ['email', 'phone number', 'social security number', 'name', 'address']
        pii_report = {}

        for col in pii_columns:
            if col in data.columns:
                # Count the number of non-null values in potential PII columns
                pii_report[col] = data[col].notnull().sum()

        return pii_report

    def anonymize_pii(self, data):
        """
        Anonymize PII data by hashing or masking it.
        Args:
            data (pd.DataFrame): Input dataset.
        Returns:
            pd.DataFrame: Dataset with anonymized PII data.
        """
        pii_columns = ['email', 'phone number', 'social security number', 'name', 'address']

        for col in pii_columns:
            if col in data.columns:
                # Apply a hash function (SHA256) to anonymize PII data
                if data[col].dtype == object:
                    data[col] = data[col].apply(lambda x: hashlib.sha256(x.encode()).hexdigest() if isinstance(x, str) else x)

        return data

    def check_for_encryption_compliance(self, data):
        """
        Check if sensitive financial data (e.g., 'ad spend', 'revenue') is encrypted or masked.
        Args:
            data (pd.DataFrame): Input dataset.
        Returns:
            dict: A report on whether sensitive columns are encrypted or masked.
        """
        encrypted_columns = ['ad spend', 'revenue']
        encryption_compliance = {}

        for col in encrypted_columns:
            if col in data.columns:
                # Check if sensitive columns have been masked or encrypted (e.g., if the value looks like an encrypted string)
                encryption_compliance[col] = data[col].apply(lambda x: isinstance(x, str) and len(x) == 64 and x.isalnum()).all()

        return encryption_compliance

    def generate_compliance_report(self, pii_report, encryption_compliance):
        """
        Generate a report summarizing the data security and compliance checks.
        Args:
            pii_report (dict): A dictionary containing identified PII columns and their counts.
            encryption_compliance (dict): A dictionary containing encryption compliance status.
        Returns:
            pd.DataFrame: A DataFrame summarizing the security and compliance check.
        """
        # Create DataFrame for PII report
        pii_df = pd.DataFrame(pii_report.items(), columns=['Column', 'PII Occurrences'])

        # Create DataFrame for encryption compliance
        encryption_df = pd.DataFrame(encryption_compliance.items(), columns=['Column', 'Encrypted'])

        # Combine both reports into a final compliance report
        compliance_report = pd.merge(pii_df, encryption_df, on="Column", how="outer")

        return compliance_report


# Initialize the DataSecurityComplianceAgent
agent = DataSecurityComplianceAgent()

# Step 1: Check for PII in the data
pii_report = agent.check_for_pii(data)

# Step 2: Anonymize the PII data
anonymized_data = agent.anonymize_pii(data)

# Step 3: Check for encryption compliance for sensitive financial data
encryption_compliance = agent.check_for_encryption_compliance(anonymized_data)

# Step 4: Generate the compliance report
compliance_report = agent.generate_compliance_report(pii_report, encryption_compliance)

# Accessing the report
print("Data Security Compliance Report:")
print(compliance_report)


Data Security Compliance Report:
     Column PII Occurrences  Encrypted
0  ad spend             NaN      False
1   revenue             NaN      False


## Agent 3: Data Lineage Tracking Agent

In [13]:
class DataLineageTrackingAgent:
    def __init__(self):
        self.name = "Data Lineage Tracking Agent"
        self.model = "gpt-4o-mini"
        self.instructions = (
            """You are a Data Lineage Tracking Agent. Your task is to trace and record the flow of data from its source
            to destination, documenting every transformation and operation applied to the data at each step.
            Track data source, transformations, and provide a detailed summary of the data lineage."""
        )
        self.lineage_log = []

    def track_data_source(self, data, source):
        """
        Track the data source.
        Args:
            data (pd.DataFrame): The dataset.
            source (str): The source from where the data is coming.
        """
        self.lineage_log.append({
            'step': 'Source',
            'description': f"Data sourced from {source}",
            'columns': list(data.columns),
            'data': data.head().to_dict()
        })

    def track_transformation(self, data, transformation, transformation_details):
        """
        Track a transformation step.
        Args:
            data (pd.DataFrame): The dataset after transformation.
            transformation (str): Name of the transformation.
            transformation_details (str): Details about the transformation.
        """
        self.lineage_log.append({
            'step': 'Transformation',
            'description': f"Transformation applied: {transformation} - {transformation_details}",
            'columns': list(data.columns),
            'data': data.head().to_dict()
        })

    def track_data_destination(self, data, destination):
        """
        Track the data destination.
        Args:
            data (pd.DataFrame): The dataset before being stored or used.
            destination (str): The destination where the data is being sent or stored.
        """
        self.lineage_log.append({
            'step': 'Destination',
            'description': f"Data sent to {destination}",
            'columns': list(data.columns),
            'data': data.head().to_dict()
        })

    def generate_lineage_report(self):
        """
        Generate a summary report of the data lineage.
        Returns:
            pd.DataFrame: A DataFrame summarizing the data lineage.
        """
        lineage_df = pd.DataFrame(self.lineage_log)
        return lineage_df

    def visualize_lineage(self):
        """
        Visualize the data lineage in a textual format.
        Returns:
            str: A textual summary of the data lineage.
        """
        lineage_text = ""
        for log in self.lineage_log:
            lineage_text += f"Step: {log['step']}\n"
            lineage_text += f"Description: {log['description']}\n"
            lineage_text += f"Columns: {', '.join(log['columns'])}\n"
            lineage_text += f"Sample Data: {log['data']}\n"
            lineage_text += "-" * 40 + "\n"
        return lineage_text

    def clean_data(self, data):
        """
        Clean the 'ad spend' and 'revenue' columns by removing non-numeric characters
        and converting them to numeric values.
        """
        # Remove '₱' symbol and commas, and convert to numeric
        for column in ['ad spend', 'revenue']:
            if column in data.columns:
                data[column] = data[column].replace({'₱': '', ',': ''}, regex=True)
                data[column] = pd.to_numeric(data[column], errors='coerce')  # Convert to numeric, invalid parsing becomes NaN
        return data

# Initialize the DataLineageTrackingAgent
agent = DataLineageTrackingAgent()

# Step 1: Track data source
agent.track_data_source(data, source="Marketing Campaign")

# Step 2: Clean the data (remove currency symbols and convert to numeric)
data = agent.clean_data(data)

# Step 3: Perform a transformation (e.g., currency conversion) and track it
data['ad spend'] = data['ad spend'] * 0.85  # Example transformation (converting currency)
agent.track_transformation(data, transformation="Currency Conversion", transformation_details="Converted ad spend to USD.")

# Step 4: Track data destination (e.g., sending to a database)
agent.track_data_destination(data, destination="Reporting Database")

# Step 5: Generate the lineage report
lineage_report = agent.generate_lineage_report()

# Step 6: Visualize the lineage (textual format)
lineage_text = agent.visualize_lineage()

# Accessing the report
print("Data Lineage Report (DataFrame):")
print(lineage_report)

print("\nData Lineage Visualization (Textual):")
print(lineage_text)


Data Lineage Report (DataFrame):
             step                                        description  \
0          Source               Data sourced from Marketing Campaign   
1  Transformation  Transformation applied: Currency Conversion - ...   
2     Destination                    Data sent to Reporting Database   

                                             columns  \
0  [date, source, medium, delivery_available, dev...   
1  [date, source, medium, delivery_available, dev...   
2  [date, source, medium, delivery_available, dev...   

                                                data  
0  {'date': {0: 2020-05-11 00:00:00, 1: 2020-05-1...  
1  {'date': {0: 2020-05-11 00:00:00, 1: 2020-05-1...  
2  {'date': {0: 2020-05-11 00:00:00, 1: 2020-05-1...  

Data Lineage Visualization (Textual):
Step: Source
Description: Data sourced from Marketing Campaign
Columns: date, source, medium, delivery_available, device_type, promo_activated, filter_used, pageviews, visits, productClick, addT